In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split #импортировали функцию для разделения датасета на обучающую и тестовую выборки

data = pd.read_csv("heart.csv")
x = data.drop("target", axis=1) #это признаки + удаляем целевой столбец, axis=1 - операция выполняется по столбцу
y = data["target"] #это целевой столбец

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42, stratify=y)  #делим на обучающую и тестовую выборку, 0.2 - размер тестовой выборки, 42 - рандом для выбора, stratify=y - чтоб целевая переменная распределилась равномерно в обучающей и тестовой выборке


In [ ]:
from sklearn.naive_bayes import GaussianNB #наивный байес
from sklearn.linear_model import LogisticRegression #лог регрессия
from sklearn.neighbors import KNeighborsClassifier #метод k-ближайших соседей

nb_model = GaussianNB() #создвм модель наивный байес 
nb_model.fit(x_train, y_train) #ообучаем

log_model = LogisticRegression(max_iter=5000) #создвм модель лог регрессии, 1000 - кол-во иитераций, 
log_model.fit(x_train, y_train) #обучаем

best_k = None #пока мы не знаем, какое k лучше всего подходит для модели, поэтому ставим None
best_score = 0 #начальное значение точности
for k in range(1, 21):  # цикл для проверки k от 1 до 20
    knn = KNeighborsClassifier(n_neighbors=k) #создаем модель
    knn.fit(x_train, y_train) #обучаем модель
    score = knn.score(x_test, y_test)  # "Метод .score() для классификаторов в sklearn возвращает accuracy (долю правильных предсказаний)"
    if score > best_score: #если текущая модель точнее предыдущих, то сохраняются показатели при этой модели
        best_score = score
        best_k = k

knn_model = KNeighborsClassifier(n_neighbors=best_k) #создаем модель с наилучшик k полученным выше
knn_model.fit(x_train, y_train) #обчаем модель


In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score #импортируем 1) матрицу ошибок, 2) точность для правильных предсказаний, 3) точность насколько предсказания класса 1 действительно верны, 4) насколько модель нашла все объекты класса 1, "гармоническое среднее между precision и recall (баланс между ними)"

models = {
    "Naive Bayes": nb_model,
    "Logistic Regression": log_model,
    "KNN": knn_model
} #словарь моделей

for name, model in models.items(): # перебираем каждую пару - название модели и объект модели
    y_pred = model.predict(x_test) #предсказания модели на тестовой выборке
    print(f"\n{name}") 
    print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred)) # "Показываем, сколько объектов модель классифицировала правильно и где ошиблась:
    print("Accuracy:", accuracy_score(y_test, y_pred)) #доля правильных предсказаний
    print("Precision:", precision_score(y_test, y_pred)) #bз всех предсказаний класса 1 сколько действительно были класса 1 "Формула: TP / (TP + FP)"
    print("Recall:", recall_score(y_test, y_pred)) #из всех настоящих объектов класса 1 сколько модель нашла "Формула: TP / (TP + FN)"
    print("F1-score:", f1_score(y_test, y_pred)) #баланс между precision и recall.

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc #roc_curve - функция для построения рок кривой, "auc — вычисляет площадь под ROC-кривой (AUC), которая показывает качество модели: чем ближе к 1, тем лучше."

for name, model in models.items(): 
    y_prob = model.predict_proba(x_test)[:,1] #вероятности принадлежности к каждому классу, берём вероятность принадлежности к классу 1
    
    fpr, tpr, _ = roc_curve(y_test, y_prob) # Строим рок кривую
    roc_auc = auc(fpr, tpr) #"auc() считает площадь под кривой — чем больше, тем лучше модель различает классы"
    
    plt.plot(fpr, tpr, label=f"{name} (AUC = {roc_auc:.2f})") #рисуем кривые и легенду

plt.plot([0,1], [0,1])

plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC-кривые для моделей")
plt.legend()
plt.show()
